In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models, regularizers

# Define the encoder
encoder_input = layers.Input(shape=(84, 1))  # Adjust the shape based on your RNA sequence length
x = layers.Conv1D(32, 7, activation="relu", padding="same")(encoder_input)
x = layers.MaxPooling1D(2, padding="same")(x)
x = layers.Conv1D(16, 7, activation="relu", padding="same")(x)
encoded = layers.MaxPooling1D(2, padding="same", name="encoder")(x)

# Define the decoder
x = layers.Conv1D(16, 7, activation="relu", padding="same")(encoded)
x = layers.UpSampling1D(2)(x)
x = layers.Conv1D(32, 7, activation="relu", padding="same")(x)
x = layers.UpSampling1D(2)(x)
decoded = layers.Conv1D(1, 7, activation="sigmoid", padding="same")(x)

# Autoencoder model
autoencoder = models.Model(encoder_input, decoded)
autoencoder.compile(optimizer='adam', loss='mean_squared_error')

In [2]:
from src.dataset import load_benchmark_dataset, Species, Modification

dataset = load_benchmark_dataset(Species.human, Modification.psi)
test_dataset = load_benchmark_dataset(Species.human, Modification.psi, True)

In [3]:
from src.features.encodings import pstnpss

encoder = binary.Encoder()

In [4]:
encoded_samples = encoder.fit_transform(dataset.samples)
encoded_test_samples = encoder.fit_transform(test_dataset.samples)
targets = dataset.targets
test_targets = test_dataset.targets

In [5]:
autoencoder.fit(encoded_samples, encoded_samples,
                epochs=100,
                batch_size=256,
                shuffle=True,
                validation_data=(encoded_test_samples, encoded_test_samples))

In [6]:
encoder_model = models.Model(inputs=autoencoder.input,
                       outputs=autoencoder.get_layer("encoder").output)

In [11]:
encoded_train_new = encoder_model.predict(encoded_samples)
encoded_test_new = encoder_model.predict(encoded_test_samples)

In [8]:
# encoded_train_new = tf.reshape(encoded_train_new, [990, -1])
# encoded_test_new = tf.reshape(encoded_test_new, [200, -1])

In [16]:
model = models.Sequential([
    layers.Input(shape=encoded_train_new.shape[1:]),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [17]:
model.fit(encoded_train_new, targets,
          epochs=100,
          batch_size=32,
          validation_data=(encoded_test_new, test_targets))

In [10]:
(classifier.predict(encoded_test_new) == test_targets.values).sum() / len(test_targets.values)

In [46]:
classifier.predict(encoded_train_new) == targets.values

In [7]:
from src.dataset import load_dataset, Species, Modification, split_balanced

x_train, x_test, y_train, y_test = split_balanced(encoded_samples, dataset.targets, test_size=0.2)
x_train, x_val, y_train, y_val = split_balanced(x_train, y_train, test_size=0.2)

In [8]:
x_train_encoded = encoder.predict(x_train)
x_val_encoded = encoder.predict(x_val)
x_test_encoded = encoder.predict(x_test)

In [14]:
x_train_encoded

In [12]:
# Classification model
model = models.Sequential([
    layers.Input(shape=x_train_encoded.shape[1:]),
    layers.Flatten(),  # Make sure to adjust the input shape
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.8),
    layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [13]:
model.fit(x_train_encoded, y_train,
          epochs=100,
          batch_size=32,
          validation_data=(x_val_encoded, y_val))